##### 토큰화

##### 형태소 분석(pos-tagging)
- 텍스트 분석을 하기 위해서 단어의 형태로 데이터를 처리해줍니다
- 단어로 만드는 과정에서 1)분석에 불필요한 불용어 제거 2)단어를 사전형으로 처리(가고 있다->
'가다', '있다')
- 명사, 형용사, 동사, 부사의 품사만 분석 대상으로 선정

In [1]:
from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np

In [2]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

In [3]:
from konlpy.tag import Okt

In [5]:
data = pd.read_csv("아이돌인스타그램_2022(전처리 및 광고제거).csv", encoding='utf-8-sig')
data.head()

,Unnamed: 0,index,time,doc,url
0,0,0,2022. 5. 9. 13:10,지수 드라마 설강화 배우 김혜윤 인스타그램에 지수 배우 김혜윤님 인스...,https://blog.naver.com/syr03051/222723113775
1,1,1,2022. 9. 29. 8:10,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 ...,https://blog.naver.com/syr03051/222885506031
2,2,2,2022. 9. 2. 14:52,지수 인스타그램 사진들 모음 지수 인스타그램 사진들 모음 ...,https://blog.naver.com/fenergy12/222864930875
3,3,3,2022. 5. 7. 1:03,게시판 로제 인스타그램을 박채영 로제,https://blog.naver.com/199651/222723356800
4,4,4,2022. 1. 15. 16:42,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 만명 팔로잉...,https://blog.naver.com/alwayshappiness211/2226...


In [6]:
#필요없는 컬럼 삭제
del data['Unnamed: 0']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7089 entries, 0 to 7088
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   7089 non-null   int64 
 1   time    7089 non-null   object
 2   doc     7089 non-null   object
 3   url     7089 non-null   object
dtypes: int64(1), object(3)
memory usage: 221.7+ KB


In [7]:
data.head()

,index,time,doc,url
0,0,2022. 5. 9. 13:10,지수 드라마 설강화 배우 김혜윤 인스타그램에 지수 배우 김혜윤님 인스...,https://blog.naver.com/syr03051/222723113775
1,1,2022. 9. 29. 8:10,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 ...,https://blog.naver.com/syr03051/222885506031
2,2,2022. 9. 2. 14:52,지수 인스타그램 사진들 모음 지수 인스타그램 사진들 모음 ...,https://blog.naver.com/fenergy12/222864930875
3,3,2022. 5. 7. 1:03,게시판 로제 인스타그램을 박채영 로제,https://blog.naver.com/199651/222723356800
4,4,2022. 1. 15. 16:42,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 만명 팔로잉...,https://blog.naver.com/alwayshappiness211/2226...


In [8]:
#원하는 태그 추출&불용어 제거
okt = Okt()

#불용어
#한국어불용어사전 불러오기
stopwords = pd.read_csv('ko-stopwords.csv')

stopwords=list(stopwords['stopwords'])
stopwords.extend(['되다', '보다', '되어다', '하다', '가다' , '받다', '드리다', '자다', '너무', '이다', '없다', '만', '이렇게', '이미지'
                  , '말', '파', '눈', '분', '네', '위', '시', '감', '전', '후', '에스', '개', '데', '코드', '명', '여', '싶다'
                  , '안', '전', '일리', '에이', '후', '백', '날', '만', '듯', '드', '님', '미국', '통해', '몰', '점'
                  , '지난', '필라테스', '여러', '포스팅' 
                  ,'인스타그램', '리사', '지수', '블핑', '제니', '로제', '박채영', '선미', '나연', '카리나'
                  , '조이', '윤아', '태연', '전소미', '산다라박', '크리스탈', '손나은', '수지', '안유진', '장원영'
                  ]) #불용어 추가
stopwords=set(stopwords)

#형태소 분석 함수 만들기
def okt_pos_tagging(string):
    pos_words = okt.pos(str(string), stem=True, norm=True)#형태소 분석, 단어는 사전형으로 바꿔주기- 'Verb', 'Adverb'
    words = [word for word, tag in pos_words if tag in ['Noun', 'Adjective'] if word not in stopwords]
    return words

In [9]:
print(stopwords)

{'그렇지 않다면', '것', '결과에 이르다', '그렇지만', '지난', '솨', '위해서', '불구하고', '까지도', '없다', '팔', '아니었다면', '막론하고', '하는것만 못하다', '이외에도', '하고있었다', '봐라', '여부', '이라면', '손나은', '결국', '지든지', '즉', '양자', '얼마큼', '백', '잠깐', '이와 같은', '고로', '다른 방면으로', '제외하고', '마음대로', '시각', '한다면', '그에 따르는', '왜', '보다더', '어느때', '나머지는', '바꾸어말하자면', '총적으로 말하면', '아니라면', '그러므로', '앗', '무렵', '명', '하자마자', '지만', '에서', '토하다', '우리들', '답다', '아이', '시키다', '에 있다', '펄렁', '어느해', '바꿔 말하면', '전소미', '논하지 않다', '꽈당', '댕그', '어찌됏어', '여러', '로', '좋아', '하는 김에', '에스', '헐떡헐떡', '곧', '삐걱거리다', '가', '전후', '기준으로', '다음', '뒤이어', '하는바', '제니', '아무도', '까지', '를', '버금', '비슷하다', '그치지 않다', '어떤', '할 생각이다', '어기여차', '기타', '않기 위하여', '점', '아홉', '하지 않는다면', '하물며', '설마', '그런 까닭에', '바로', '대하면', '포스팅', '이유만으로', '정도에 이르다', '다음으로', '어느곳', '조차도', '뒤따라', '로 인하여', '월', '어찌하든지', '들', '한마디', '각종', '그만이다', '일반적으로', '아이구', '공동으로', '가다', '요만한 것', '관계가 있다', '하기는한데', '반대로 말하자면', '만약에', '마저', '아무거나', '저것만큼', '하는것도', '듯', '얼마든지', '요만한걸', '각자', '말할것도 없고', '할뿐', '하기 위하여', '전부', '왜냐하면', '하지 않도록', '할수있어', '

In [10]:
#형태소 분석 적용
tokenizing_doc=[]
for i in tqdm(data['doc']):
    tokenizing_doc.append(okt_pos_tagging(i))

#새로운 tokenizing이라는 컬럼에 키워드 형태로 만든 토큰?  
data['token']=tokenizing_doc

100%|██████████| 7089/7089 [02:58<00:00, 39.66it/s]


In [11]:
data.doc

0        지수 드라마 설강화 배우 김혜윤 인스타그램에    지수     배우 김혜윤님 인스...
1        블핑 인스타 트위터    리사 인스타그램     리사 인스타그램 지수 댓글    ...
2           지수 인스타그램 사진들 모음        지수 인스타그램 사진들 모음    ...
3                         게시판    로제 인스타그램을 박채영        로제 
4        로제 인스타그램    로제 인스타그램 스토리 포에버       팔로워 만명 팔로잉...
                              ...                        
7084     크리스탈 폴로   셔츠   포니 가방 캐주얼룩 완성 걸스픽     크리스탈 폴로 ...
7085     스타일링 따라하고 싶은 셀럽들의 폴로 랄프로렌 셔츠 스타일링 크리스탈 기은세 꾸리...
7086     게시판 크리스탈 정수정 근황 좋아보여요 광고실행사     배우 정수정 및 크리스탈...
7087     스타패션 크리스탈 정수정 인스타그램 스트라이프 럭비 티셔츠 폴로 랄프로렌     ...
7088     패션정보 셀럽들이 사랑한 크리스탈헤이즈 하리보 목걸이 모음 히요정       안녕...
Name: doc, Length: 7089, dtype: object

In [12]:
data.token

0                            [드라마, 설강화, 배우, 김혜윤, 배우, 김혜윤]
1                                [인스타, 트위터, 댓글, 프루덴셜, 센터]
2                        [사진, 모음, 사진, 모음, 사진, 모음, 사진, 모음]
3                                                   [게시판]
4                             [스토리, 포에버, 팔로워, 팔로잉, 게시, 물]
                              ...                        
7084    [폴로, 셔츠, 포니, 가방, 얼룩, 완성, 걸스, 픽, 폴로, 셔츠, 포니, 가방...
7085    [스타, 일링, 셀럽, 폴로, 랄프, 로렌, 셔츠, 스타, 일링, 기은세, 꾸리, ...
7086    [게시판, 정수정, 근황, 좋다, 광고, 실, 행사, 배우, 정수정, 님근황, 해바...
7087    [스타, 패션, 정수정, 스트라이프, 럭비, 티셔츠, 폴로, 랄프, 로렌, 안녕하다...
7088    [패션, 정보, 셀럽, 사랑, 헤이즈, 하리보, 목걸이, 모음, 요정, 안녕하다, ...
Name: token, Length: 7089, dtype: object

In [13]:
#명사 추출 
text_noun=[okt.nouns(doc) for doc in data.doc]
data['text_noun'] = text_noun
data['text_noun'] = data.text_noun.apply(lambda x: [a for a in x if a not in stopwords])

In [14]:
#필요없는 컬럼 삭제
#del data['Unnamed: 0']
#data.info()

In [15]:
data.head()

,index,time,doc,url,token,text_noun
0,0,2022. 5. 9. 13:10,지수 드라마 설강화 배우 김혜윤 인스타그램에 지수 배우 김혜윤님 인스...,https://blog.naver.com/syr03051/222723113775,"[드라마, 설강화, 배우, 김혜윤, 배우, 김혜윤]","[드라마, 설강화, 배우, 김혜윤, 배우, 김혜윤]"
1,1,2022. 9. 29. 8:10,블핑 인스타 트위터 리사 인스타그램 리사 인스타그램 지수 댓글 ...,https://blog.naver.com/syr03051/222885506031,"[인스타, 트위터, 댓글, 프루덴셜, 센터]","[인스타, 트위터, 댓글, 프루덴셜, 센터]"
2,2,2022. 9. 2. 14:52,지수 인스타그램 사진들 모음 지수 인스타그램 사진들 모음 ...,https://blog.naver.com/fenergy12/222864930875,"[사진, 모음, 사진, 모음, 사진, 모음, 사진, 모음]","[사진, 모음, 사진, 모음, 사진, 모음, 사진, 모음]"
3,3,2022. 5. 7. 1:03,게시판 로제 인스타그램을 박채영 로제,https://blog.naver.com/199651/222723356800,[게시판],[게시판]
4,4,2022. 1. 15. 16:42,로제 인스타그램 로제 인스타그램 스토리 포에버 팔로워 만명 팔로잉...,https://blog.naver.com/alwayshappiness211/2226...,"[스토리, 포에버, 팔로워, 팔로잉, 게시, 물]","[스토리, 포에버, 팔로워, 팔로잉, 게시, 물]"


##### 토큰화까지 끝난 파일 저장

In [16]:
data.to_csv("data_토큰화.csv", encoding='utf-8-sig')
data.to_pickle('data_토큰화.pkl')